In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
coo_matrix

scipy.sparse.coo.coo_matrix

In [2]:
df = pd.read_csv("PA-MDCS-graph.csv")
#df['GEOID101'] = np.array(df['GEOID101'],dtype=np.int64)
#df['GEOID102'] = np.array(df['GEOID102'],dtype=np.int64)
print len(df)
df = df.dropna()
df.GEOID102 = df.GEOID102.astype(np.int64())
print len(df)
df

6991
6517


,GEOID101,GEOID102,st_geometr
0,4200323616,4200339624,ST_MultiLineString
1,4200323616,4200353136,ST_MultiLineString
2,4200324160,4200361000,ST_MultiLineString
3,4200324160,4200369584,ST_MultiLineString
4,4200324160,4200369776,ST_MultiLineString
5,4200325400,4200327608,ST_MultiLineString
6,4200325400,4200332832,ST_MultiLineString
7,4200325400,4200376104,ST_MultiLineString
8,4200325400,4200382800,ST_MultiLineString
9,4200325400,4201914320,ST_MultiLineString


In [3]:
df.iloc[4,1]

4200369776

In [4]:
nodes1 = df.iloc[:,0].unique()
nodes2 = df.iloc[:,1].unique()
nodes = set(nodes1.tolist()+nodes2.tolist())

In [5]:
len(nodes)

2574

In [6]:
import csv
import os

In [7]:
os.getcwd()
with open('../list_of_precincts.txt','r') as f:
    reader = csv.reader(f)
    a = list(reader)
plist = [b[0].strip() for b in a]


In [8]:
dbf = open('../data/Penn MCDS Data/MCDS.dbf','r').read()

dbf[4000:40020]
out = dbf.rsplit("Findlay")
len(out)


3

In [9]:
geoid_d = {}
import re
dbfu = dbf.upper()
for precinct in plist:
#    print precinct
    #out = dbfu.rsplit(precinct.upper() + '    ')
    out = re.split('([0-9])' + precinct.upper() + '    ', dbfu)
    if len(out) != 1:
        geoid_d[precinct] = out[0][-9:] + out[1][0:2]
    else:
        print 'precinct ' + precinct + ' not found'

In [10]:
aa = dbfu.rsplit(precinct.upper())
print aa[0][-10:]


4202985352


In [11]:
geo_id_rev = {int(vals): keys for keys, vals in geoid_d.iteritems()}
print len(geo_id_rev) - len(geoid_d)
geo_id_rev[4202984192]

0


'West Whiteland'

In [12]:
all_geo_ids = np.array(geo_id_rev.keys())
gid_adj_dict = {key: idx for idx,key in enumerate(all_geo_ids)}
row_list = []
col_list = []
data_list = []

for key in geo_id_rev.keys():
    
    out1 = df.loc[df['GEOID102'] == key]
    
    
    
    row_list.extend(np.array(out1['GEOID101']))
    col_list.extend(np.array(out1['GEOID102']))
    data_list.extend(np.ones(len(out1)))
    
    out2 = df.loc[df['GEOID101'] == key]
    row_list.extend(np.array(out2['GEOID101']))
    col_list.extend(np.array(out2['GEOID102']))
    

removed_edges = [idx for idx,row in enumerate(row_list) 
                 if row not in all_geo_ids or col_list[idx] not in all_geo_ids]
kept_edges = [idx for idx in range(len(row_list)) if idx not in removed_edges]
    
row_list2 = np.array([gid_adj_dict[row_list[edge]] for edge in kept_edges],dtype=np.int64)
col_list2 = np.array([gid_adj_dict[col_list[edge]] for edge in kept_edges],dtype=np.int64)
data_list =np.ones_like(col_list2)


In [13]:
len(row_list2)
len(col_list2)
print len(data_list)
print len(all_geo_ids)

230
72


In [14]:
Adj_mat = coo_matrix((np.array(data_list), (row_list2, col_list2)),shape=(len(all_geo_ids),len(all_geo_ids)))
Adj_mat = Adj_mat + Adj_mat.T/2

#import matplotlib.pylab as plt
#plt.spy(Adj_mat)
#plt.show()


/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:2699: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


In [80]:
import random
import scipy.sparse as sparse
beta = 1


total_pop = 100
max_pop = np.inf
def in_district_edges(district, adjacency_matrix, node):
    return len(set(adjacency_matrix[node,:].nonzero()[1]).intersection(district))

def update_markov_chain(cur_maps, district_pops, mcds_pops,  adjacency_matrix):
    num_districts = len(cur_maps)
    ideal_pop = total_pop*1.0/num_districts*np.ones([num_districts])
    
    assigned_mcds = [mcds for dist in cur_maps for mcds in dist]
    
    rand_mcds = random.choice(assigned_mcds)
    cur_dist = [k for k in range(len(cur_maps)) if rand_mcds in cur_maps[k]][0]
    rand_neighbor = random.choice(adjacency_matrix[rand_mcds,:].nonzero()[1])
    
    if rand_neighbor in assigned_mcds:
        cur_neighbor_dist = [k for k in range(len(cur_maps)) if rand_neighbor in cur_maps[k]][0]
        new_dist_pops = district_pops[:]
        new_dist_pops[cur_dist] += mcds_pops[rand_neighbor]
        
        cur_bonds = in_district_edges(cur_maps[cur_dist], 
                                        adjacency_matrix, rand_neighbor)*1.0
        new_bonds = in_district_edges(cur_maps[cur_neighbor_dist], adjacency_matrix, rand_neighbor)*1.0
        if new_bonds == 0:
            cost_switch = 1
        else:
            cost_switch = (
                        np.exp(-beta*(np.linalg.norm(new_dist_pops - ideal_pop) - 
                                      np.linalg.norm(district_pops-ideal_pop))))
        if np.random.rand() < cost_switch:
            cur_maps[cur_neighbor_dist] = [mcds for mcds in cur_maps[cur_neighbor_dist] if mcds != rand_neighbor]
            district_pops[cur_neighbor_dist] -= mcds_pops[rand_neighbor]
            cur_maps[cur_dist].append(rand_neighbor)
            district_pops[cur_dist] += mcds_pops[rand_neighbor]
            return cur_maps, district_pops
        else:
            return cur_maps, district_pops
    elif  district_pops[cur_dist] + mcds_pops[rand_neighbor] < max_pop:
        cur_maps[cur_dist].append(rand_neighbor)
        district_pops[cur_dist] += mcds_pops[rand_neighbor]
        return cur_maps, district_pops
    else:
        return cur_maps, district_pops

def run_markov_chain( cur_maps, district_pops, mcds_pops, adjacency_matrix):
    # districts will be a list of lists (of indices) saying which MCDS belong to which list
    # district_pops is an np.arra
    # adjacency matrix will show the graph topology
    num_its = 1000
    for it in range(num_its):
        cur_maps, district_pops = update_markov_chain(cur_maps, district_pops, mcds_pops, adjacency_matrix)
    return cur_maps

In [81]:
cur_maps = [[1, 10, 12], [4], [15]]
district_pops = np.array([100, 100, 100])
mcds_pops = np.random.randn((len(all_geo_ids)))
new_maps = run_markov_chain(cur_maps, district_pops, mcds_pops, Adj_mat)
print new_maps

[[48, 68, 1, 50, 32, 47, 35, 52, 62, 20, 71, 49, 31, 41, 37, 14, 5, 53, 70, 2, 60, 10, 18], [24, 34, 45, 36, 27, 44, 61, 11, 7, 4, 9, 28, 63, 39, 21, 65, 22, 12, 42, 6, 15, 0, 51, 17, 26, 23, 30, 29, 16, 25, 55, 58, 59, 56, 66], []]


In [43]:
print rand_mcds
print rand_neighbor
print Adj_mat[rand_mcds,:]
Adj_mat[0,:].nonzero()[1]

10
52
  (0, 35)	2.0
  (0, 52)	2.0
  (0, 60)	2.0


array([12, 16, 22, 23, 30, 55, 70], dtype=int32)

In [45]:
len()set(Adj_mat[0,:].nonzero()[1]).intersection(cur_maps[0])

1

In [51]:
Adj_mat[4,:].nonzero()[1]

array([ 9, 15, 29], dtype=int32)

In [60]:
np.where(rand_mcds in cur_maps)


[1]

In [23]:
print np.linalg.norm( )

TypeError: data type not understood

In [28]:
a = np.array([20,40,40])
print a
b = np.ones_like(a)
print b
np.linalg.norm(a - b)

[20 40 40]
[1 1 1]


58.33523806414096

In [30]:
np.ones([3])

array([ 1.,  1.,  1.])

In [33]:
np.random.rand()

0.46729786842658494